In [24]:
import torch
import torch.nn as nn
import numpy as np

# Sample dataset (Food, Travel, Entertainment, Utilities)
data = np.array([
    [5000, 2000, 1000, 1500],
    [5200, 1800, 1200, 1600],
    [4800, 2100,  900, 1700],
    [5100, 2200, 1100, 1550],
    [5300, 2400, 1200, 1650]
], dtype=np.float32)

In [25]:
# Normalize data (important for NN training)
data = (data - data.mean(axis=0)) / data.std(axis=0)
#data.std(axis=0) computes the standard deviation of each column (feature) in the data NumPy array

# Prepare sequences (last 3 months -> predict next month) if less than 3 months take len(data)-1
seq_length = min(3, len(data)-1)

X, y = [], []
for i in range(len(data) - seq_length):
    X.append(data[i:i+seq_length])   # past 3 months
    y.append(data[i+seq_length])     # next month

X = np.array(X)  # shape: (samples, seq_len, features)
y = np.array(y)  # shape: (samples, features)

# Convert to tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [26]:
X,y

(tensor([[[-0.4650, -0.5000, -0.6860, -1.4142],
          [ 0.6975, -1.5000,  1.0290,  0.0000],
          [-1.6275,  0.0000, -1.5435,  1.4142]],
 
         [[ 0.6975, -1.5000,  1.0290,  0.0000],
          [-1.6275,  0.0000, -1.5435,  1.4142],
          [ 0.1162,  0.5000,  0.1715, -0.7071]]]),
 tensor([[ 0.1162,  0.5000,  0.1715, -0.7071],
         [ 1.2787,  1.5000,  1.0290,  0.7071]]))

In [27]:
# Define LSTM model
class ExpensePredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ExpensePredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]   # take last time step
        out = self.fc(out)
        return out


In [28]:
# Model params
input_size = 4      # categories: Food, Travel, Entertainment, Utilities
hidden_size = 16
output_size = 4     # predict all 4 categories

expense_predictor_model = ExpensePredictor(input_size, hidden_size, output_size)

In [29]:
# Loss & optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(expense_predictor_model.parameters(), lr=0.01)


MSELoss → minimizes squared error between predicted and actual expenses.

Adam optimizer → updates weights.

We train for 200 epochs.

Every iteration, the model learns to map “past 3 months → next month”.

In [30]:
# Training loop
epochs = 200
for epoch in range(epochs):
    expense_predictor_model.train()
    output = expense_predictor_model(X)
    loss = criterion(output, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [50/200], Loss: 0.0015
Epoch [100/200], Loss: 0.0000
Epoch [150/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000


In [31]:
with torch.no_grad():
    last_seq = torch.tensor(data[-3:], dtype=torch.float32).unsqueeze(0)  # shape (1, seq_len, features)
    prediction = expense_predictor_model(last_seq)
    print("Next month prediction (normalized):", prediction.numpy())

Next month prediction (normalized): [[1.4016656 1.5059112 1.1555291 0.8338667]]


In [32]:
# Denormalize prediction
mean = np.array([5000, 2100, 1100, 1600])  # example mean (use actual mean from training)
std = np.array([150, 200, 100, 80])        # example std (use actual std from training)

real_prediction = prediction.numpy() * std + mean
print("Predicted expenses (Food, Travel, Entertainment, Utilities):")
print(real_prediction)


Predicted expenses (Food, Travel, Entertainment, Utilities):
[[5210.24983525 2401.18224621 1215.55291414 1666.70933723]]
